In [1]:
%%bash
pip install -q transformers
pip install seqeval

  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=7955e87989d1dce62451c6f90d43959f5c2e4b45ece8b69b6881c70664361c7f
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval


In [3]:
!wget http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_pt.tar.gz

--2020-04-06 03:23:52--  http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_pt.tar.gz
Resolving files.deeppavlov.ai (files.deeppavlov.ai)... 93.175.29.74
Connecting to files.deeppavlov.ai (files.deeppavlov.ai)|93.175.29.74|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 662024852 (631M) [application/octet-stream]
Saving to: ‘rubert_cased_L-12_H-768_A-12_pt.tar.gz’

rubert_cased_L-12_H 100%[===================>] 631.36M  4.29MB/s    in 2m 13s  

2020-04-06 03:26:07 (4.74 MB/s) - ‘rubert_cased_L-12_H-768_A-12_pt.tar.gz’ saved [662024852/662024852]



In [5]:
!tar -xvf rubert_cased_L-12_H-768_A-12_pt.tar.gz

rubert_cased_L-12_H-768_A-12_pt/
rubert_cased_L-12_H-768_A-12_pt/bert_config.json
rubert_cased_L-12_H-768_A-12_pt/vocab.txt
rubert_cased_L-12_H-768_A-12_pt/pytorch_model.bin


In [6]:
!ls -lr

total 646528
drwxr-xr-x 1 root root      4096 Mar 18 16:23 sample_data
-rw-r--r-- 1 root root 662024852 Feb 18 14:45 rubert_cased_L-12_H-768_A-12_pt.tar.gz
drwxr-xr-x 2 1000 1000      4096 Feb 18 14:41 rubert_cased_L-12_H-768_A-12_pt
drwx------ 4 root root      4096 Apr  6 03:23 drive


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path 

import os

import torch
import torch.optim as optim
import copy
import random 

# transformers
import transformers
from transformers import BertModel, BertTokenizer, BertConfig
from transformers import RobertaModel, RobertaTokenizer, RobertaConfig

In [44]:
train = pd.read_csv('train_v1.csv')
dev = pd.read_csv('dev_v1.csv')
test = pd.read_csv('test_v1.csv')
print(train.shape, dev.shape, test.shape)

(6001, 2) (609, 2) (1522, 2)


In [45]:
train.head(2)

,tweet,class
0,И оксикодона На эту часть я точно не жалуюсь,0
1,Маргинал колющийся крокодилом через дорогу или...,0


In [0]:
x_train = list(train['tweet'])
y_train = list(train['class'])

x_dev = list(dev['cleaned_tweet'])
y_dev = list(dev['class'])

x_test = list(test['cleaned_tweet'])
y_test = list(test['class'])

In [0]:
MAX_LEN = 200
bs = 8
model = 'rubert'
model_name = 'rubert_cased_L-12_H-768_A-12_pt'#'biobert_large' #roberta-large

In [56]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print(device)

cuda


In [57]:
if model == 'rubert':
  tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case=True)
else:
  tokenizer = RobertaTokenizer.from_pretrained(model_name, do_lower_case=True)
print(tokenizer.cls_token, tokenizer.cls_token_id)
print(tokenizer.pad_token, tokenizer.pad_token_id)
print(tokenizer.sep_token, tokenizer.sep_token_id)
print(tokenizer.unk_token, tokenizer.unk_token_id)

[CLS] 101
[PAD] 0
[SEP] 102
[UNK] 100


In [0]:
def tokenize_list(lst, tokenizer):
  tmp = []
  for i in lst:
    if model == 'rubert':
      tokens = tokenizer.tokenize(i)
    else:
      tokens = tokenizer.tokenize(i, add_prefix_space=True)
    tokens = [tokenizer.cls_token] + tokens + [tokenizer.sep_token]
    tmp.append(tokens)
  return tmp

In [59]:
x_train_tokenized = tokenize_list(x_train, tokenizer)
x_dev_tokenized = tokenize_list(x_dev, tokenizer)
x_test_tokenized = tokenize_list(x_test, tokenizer)
#x_test2_tokenized = tokenize_list(x_test2, tokenizer)
print(len(x_train_tokenized), len(x_dev_tokenized), len(x_test_tokenized))#, len(x_test2_tokenized))
print(x_train[0])
print(" ".join(x_train_tokenized[0]))

6001 609 1522
И оксикодона На эту часть я точно не жалуюсь
[CLS] и окс ##ико ##дона на эту часть я точно не жал ##ую ##сь [SEP]


In [60]:
from keras.preprocessing.sequence import pad_sequences

pad_value = tokenizer.pad_token_id
train_input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in x_train_tokenized],
                          maxlen=MAX_LEN, dtype="long", truncating="post", 
                          padding="post", value = pad_value)
dev_input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in x_dev_tokenized],
                          maxlen=MAX_LEN, dtype="long", truncating="post", 
                          padding="post", value = pad_value)
test_input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in x_test_tokenized],
                          maxlen=MAX_LEN, dtype="long", truncating="post", 
                          padding="post", value = pad_value)

print(train_input_ids.shape, dev_input_ids.shape, test_input_ids.shape)

(6001, 200) (609, 200) (1522, 200)


In [61]:
pad_value = tokenizer.pad_token_id
train_attention_masks = np.array([[int(i != pad_value)for i in ii] for ii in train_input_ids])
dev_attention_masks = np.array([[int(i != pad_value) for i in ii] for ii in dev_input_ids])
test_attention_masks = np.array([[int(i != pad_value)for i in ii] for ii in test_input_ids])
print(train_attention_masks.shape, dev_attention_masks.shape, test_attention_masks.shape)

(6001, 200) (609, 200) (1522, 200)


In [62]:
train_attention_masks[0]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0])

In [63]:
train_segment_ids = np.array([[0]*MAX_LEN for ii in train_input_ids])
dev_segment_ids = np.array([[0]*MAX_LEN for ii in dev_input_ids])
test_segment_ids = np.array([[0]*MAX_LEN for ii in test_input_ids])

print(train_segment_ids.shape, dev_segment_ids.shape, test_segment_ids.shape)

(6001, 200) (609, 200) (1522, 200)


In [0]:
tr_inputs = torch.tensor(train_input_ids)
val_inputs = torch.tensor(dev_input_ids)
te_inputs = torch.tensor(test_input_ids)

tr_masks = torch.tensor(train_attention_masks)
val_masks = torch.tensor(dev_attention_masks)
te_masks = torch.tensor(test_attention_masks)

tr_segment = torch.tensor(train_segment_ids)
val_segment = torch.tensor(dev_segment_ids)
te_segment = torch.tensor(test_segment_ids)

tr_class = torch.tensor(y_train)
val_class = torch.tensor(y_dev)
te_class = torch.tensor(y_test)

In [0]:
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)#, tr_class)
train_data = TensorDataset(tr_inputs, tr_masks, tr_segment, tr_class)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

#valid_data = TensorDataset(val_inputs, val_masks, val_tags)#, val_class)
valid_data = TensorDataset(val_inputs, val_masks, val_segment, val_class)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

#test_data = TensorDataset(te_inputs, te_masks, te_tags)#, te_class)
test_data = TensorDataset(te_inputs, te_masks, te_segment, te_class)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=bs)


In [0]:
from torch.nn.utils.rnn import pad_sequence

class SequenceClassifier(torch.nn.Module):
  def __init__(self, transformer_model, config):
    super().__init__()
    self.transformer = transformer_model
    self.out = torch.nn.Linear(config.hidden_size*2, 2)
    self.dropout = torch.nn.Dropout(config.hidden_dropout_prob)
  
  def forward(self, input_ids, attention_mask, segment_id, classification_labels=None):
    
    #Batch max length
    max_length = (attention_mask != 0).max(0)[0].nonzero()[-1].item()+1

    if max_length < input_ids.shape[1]:
      input_ids = input_ids[:, :max_length]
      attention_mask = attention_mask[:, :max_length]

    segment_id = torch.zeros_like(attention_mask)
    hidden = self.transformer(input_ids = input_ids,attention_mask = attention_mask, 
                              token_type_ids = segment_id)
    #Aggregating all the hidden states
    token_hidden = hidden[2]
    token_hidden = torch.mean(torch.mean(torch.stack(token_hidden), dim=0), 
                              dim=1)

    classifier_hidden = hidden[1]
    hidden_cat = token_hidden
    hidden_cat = torch.cat([token_hidden, classifier_hidden], dim=1)

    classification_logits = self.out(self.dropout(hidden_cat))
    outputs = [classification_logits]
    if classification_labels is not None:
      loss_fct_classification = torch.nn.CrossEntropyLoss()

      loss_classification = loss_fct_classification(classification_logits.view(-1, 2), 
                                                      classification_labels.view(-1))

      outputs += [loss_classification]
    return outputs

In [31]:
# !ls -lr rubert_cased_L-12_H-768_A-12_pt
# !cp rubert_cased_L-12_H-768_A-12_pt/bert_config.json rubert_cased_L-12_H-768_A-12_pt/config.json
# !ls -lr rubert_cased_L-12_H-768_A-12_pt

total 699192
-rw-r--r-- 1 1000 1000   1649718 Jan 31  2019 vocab.txt
-rw-r--r-- 1 1000 1000 714313247 Jan 23 13:26 pytorch_model.bin
-rw-r--r-- 1 1000 1000       521 Feb  6  2019 bert_config.json
total 699196
-rw-r--r-- 1 1000 1000   1649718 Jan 31  2019 vocab.txt
-rw-r--r-- 1 1000 1000 714313247 Jan 23 13:26 pytorch_model.bin
-rw-r--r-- 1 root root       521 Apr  6 03:31 config.json
-rw-r--r-- 1 1000 1000       521 Feb  6  2019 bert_config.json


In [0]:
if model == 'rubert':
  config = BertConfig.from_pretrained(model_name)
else:
  config = RobertaConfig.from_pretrained(model_name)
config.output_hidden_states = True

In [0]:
if model_name == 'rubert':
  transformer_model = BertModel.from_pretrained(model_name, config=config)
else:
  transformer_model = RobertaModel.from_pretrained(model_name, config=config)

In [0]:
model = SequenceClassifier(transformer_model, config)

In [0]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
model.cuda();

In [0]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=2e-5)

In [0]:
from seqeval.metrics import f1_score

def flat_accuracy_classification(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [73]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,confusion_matrix

epochs = 10
max_grad_norm = 1.0
model.zero_grad()
best_f1 = 0
for _ in tqdm(range(epochs)):
  # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for batch in tqdm(train_dataloader):

      batch = tuple(t.to(device) for t in batch)
      b_input_ids, b_input_mask, b_segment, b_classes = batch
      
      outputs = model(b_input_ids, b_input_mask, b_segment, b_classes)
      
      loss = outputs[1]
      
      loss.backward()

      tr_loss += loss.item()
      nb_tr_examples += b_input_ids.size(0)
      nb_tr_steps += 1

      torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), 
                                     max_norm=max_grad_norm)

      optimizer.step()
      model.zero_grad()

    print("Train loss: {}".format(tr_loss/nb_tr_steps))

    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy_class = 0, 0
    
    nb_eval_steps, nb_eval_examples = 0, 0
    class_preds ,class_true_labels = [], []
    

    for batch in valid_dataloader:
      batch = tuple(t.to(device) for t in batch)
      b_input_ids, b_input_mask, b_segment, b_classes = batch

      with torch.no_grad():
        outputs = model(b_input_ids, b_input_mask, b_segment, b_classes)
        eval_loss = classification_loss = outputs[1]

      classification_logits = outputs[0].detach().cpu().numpy()
      class_preds.extend(list(np.argmax(classification_logits,axis=1)))
      
      class_label_ids = b_classes.to('cpu').numpy()
      
      class_true_labels.append(class_label_ids)
      
      tmp_eval_accuracy_class = flat_accuracy_classification(classification_logits, 
                                              class_label_ids)
      eval_loss += eval_loss.mean().item()

      eval_accuracy_class += tmp_eval_accuracy_class
      nb_eval_examples += b_input_ids.size(0)
      nb_eval_steps += 1
    eval_loss = eval_loss/nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy Classifier: {}".format(eval_accuracy_class/nb_eval_steps))
    valid_tags = [l_i for l in class_true_labels for l_i in l ]
    pred_tags = [p for p in class_preds]
    print("Val F1 ",f1_score(valid_tags, pred_tags))
    print("Val Precision ",precision_score(valid_tags, pred_tags))
    print("Val Recall ",recall_score(valid_tags, pred_tags))
    print("Val Accuracy ",accuracy_score(valid_tags, pred_tags))
    print("Confusion matrix \n", confusion_matrix(valid_tags, pred_tags))
    if f1_score(valid_tags, pred_tags) >= best_f1:
      best_f1 = f1_score(valid_tags, pred_tags)
      print("Saving this model")
      torch.save(model.state_dict(), './best_model_ru_state_dict')
      torch.save(model, './best_model_ru')
      print("Copying to G drive")
      !cp ./best_model_ru_state_dict drive/My\ Drive/
      !cp ./best_model_ru drive/My\ Drive/
      print("Copied to G drive")
      




  0%|          | 0/10 [00:00<?, ?it/s]


  0%|          | 0/751 [00:00<?, ?it/s]


  0%|          | 1/751 [00:00<02:56,  4.24it/s]


  0%|          | 2/751 [00:00<02:45,  4.54it/s]


  0%|          | 3/751 [00:00<02:42,  4.62it/s]


  1%|          | 4/751 [00:00<02:35,  4.79it/s]


  1%|          | 5/751 [00:01<02:33,  4.87it/s]


  1%|          | 6/751 [00:01<02:32,  4.89it/s]


  1%|          | 7/751 [00:01<02:32,  4.87it/s]


  1%|          | 8/751 [00:01<02:31,  4.92it/s]


  1%|          | 9/751 [00:01<02:24,  5.13it/s]


  1%|▏         | 10/751 [00:01<02:20,  5.27it/s]


  1%|▏         | 11/751 [00:02<02:17,  5.38it/s]


  2%|▏         | 12/751 [00:02<02:14,  5.49it/s]


  2%|▏         | 13/751 [00:02<02:14,  5.47it/s]


  2%|▏         | 14/751 [00:02<02:13,  5.52it/s]


  2%|▏         | 15/751 [00:02<02:15,  5.43it/s]


  2%|▏         | 16/751 [00:03<02:15,  5.42it/s]


  2%|▏         | 17/751 [00:03<02:12,  5.54it/s]


  2%|▏         | 18/751 [00:03<02:13,  5.50it/s]


  3%|▎

Train loss: 0.4393372700590586
Validation loss: 0.001386763178743422
Validation Accuracy Classifier: 0.8198051948051948
Val F1  0.36571428571428577
Val Precision  0.26229508196721313
Val Recall  0.6037735849056604
Val Accuracy  0.8177339901477833
Confusion matrix 
 [[466  90]
 [ 21  32]]
Saving this model


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type SequenceClassifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Copying to G drive




 10%|█         | 1/10 [03:04<27:42, 184.72s/it]


  0%|          | 0/751 [00:00<?, ?it/s]




...

Train loss: 0.03093777937732746




 70%|███████   | 7/10 [17:52<07:36, 152.29s/it]


  0%|          | 0/751 [00:00<?, ?it/s]

Validation loss: 6.068836455597193e-07
Validation Accuracy Classifier: 0.9204545454545454
Val F1  0.2898550724637681
Val Precision  0.625
Val Recall  0.18867924528301888
Val Accuracy  0.9195402298850575
Confusion matrix 
 [[550   6]
 [ 43  10]]





  0%|          | 1/751 [00:00<02:27,  5.10it/s]


  0%|          | 2/751 [00:00<02:22,  5.27it/s]


  0%|          | 3/751 [00:00<02:19,  5.38it/s]


  1%|          | 4/751 [00:00<02:17,  5.43it/s]


  1%|          | 5/751 [00:00<02:12,  5.64it/s]


  1%|          | 6/751 [00:01<02:19,  5.35it/s]


  1%|          | 7/751 [00:01<02:20,  5.29it/s]


  1%|          | 8/751 [00:01<02:17,  5.40it/s]


  1%|          | 9/751 [00:01<02:14,  5.51it/s]


  1%|▏         | 10/751 [00:01<02:11,  5.64it/s]


  1%|▏         | 11/751 [00:01<02:13,  5.53it/s]


  2%|▏         | 12/751 [00:02<02:15,  5.44it/s]


  2%|▏         | 13/751 [00:02<02:09,  5.69it/s]


  2%|▏         | 14/751 [00:02<02:09,  5.69it/s]


  2%|▏         | 15/751 [00:02<02:11,  5.61it/s]


  2%|▏         | 16/751 [00:02<02:13,  5.51it/s]


  2%|▏         | 17/751 [00:03<02:10,  5.63it/s]


  2%|▏         | 18/751 [00:03<02:10,  5.63it/s]


  3%|▎         | 19/751 [00:03<02:11,  5.55it/s]


  3%|▎         | 20/751 [00:03<02:16,

KeyboardInterrupt: ignored

In [0]:
model = torch.load('./best_model_ru')
for name,param in model.named_parameters():
  param.requires_grad = False
model.cuda();

In [75]:
model.eval()
eval_loss, eval_accuracy_class = 0, 0

nb_eval_steps, nb_eval_examples = 0, 0
class_preds ,class_true_labels = [], []

for batch in test_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_segment, b_classes = batch

    with torch.no_grad():
      outputs = model(b_input_ids, b_input_mask, b_segment, b_classes)
      eval_loss = classification_loss = outputs[1]

    classification_logits = outputs[0].detach().cpu().numpy()
    class_preds.extend(list(np.argmax(classification_logits,axis=1)))
    
    class_label_ids = b_classes.to('cpu').numpy()
    
    class_true_labels.append(class_label_ids)
    
    tmp_eval_accuracy_class = flat_accuracy_classification(classification_logits, 
                                            class_label_ids)
    eval_loss += eval_loss.mean().item()

    eval_accuracy_class += tmp_eval_accuracy_class
    nb_eval_examples += b_input_ids.size(0)
    nb_eval_steps += 1
eval_loss = eval_loss/nb_eval_steps
print("Validation loss: {}".format(eval_loss))
print("Validation Accuracy Classifier: {}".format(eval_accuracy_class/nb_eval_steps))
valid_tags = [l_i for l in class_true_labels for l_i in l ]
pred_tags = [p for p in class_preds]
print("Val F1 ",f1_score(valid_tags, pred_tags))
print("Val Precision ",precision_score(valid_tags, pred_tags))
print("Val Recall ",recall_score(valid_tags, pred_tags))
print("Val Accuracy ",accuracy_score(valid_tags, pred_tags))
print("Confusion matrix \n", confusion_matrix(valid_tags, pred_tags))


Validation loss: 4.4737811549566686e-05
Validation Accuracy Classifier: 0.8965968586387435
Val F1  0.4233576642335766
Val Precision  0.41134751773049644
Val Recall  0.43609022556390975
Val Accuracy  0.8961892247043364
Confusion matrix 
 [[1306   83]
 [  75   58]]
